In [1]:
import numpy as np
import os
from pydrake.common import temp_directory, FindResourceOrThrow
from pydrake.geometry import (
    MeshcatVisualizer, MeshcatVisualizerParams, 
    Role, StartMeshcat
)

from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import ModelVisualizer

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


# MODEL VIEWING

In [3]:
iiwa7_model_url = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
iiwa7_model_url

'/home/dema/PycharmProjects/robot_manipulator/lib/python3.10/site-packages/pydrake/common/../lib/../share/drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf'

In [4]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(file_name=iiwa7_model_url)

[ModelInstanceIndex(2)]

In [5]:
# When this notebook is run in test mode it needs to stop execution without
# user interaction. For interactive model visualization you won't normally
# need the 'loop_once' flag.
test_mode = True if "TEST_SRCDIR" in os.environ else False

# Start the interactive visualizer.
# Click the "Stop Running" button in MeshCat when you're finished.
visualizer.Run(loop_once=test_mode)



Click 'Stop Running' or press Esc to quit


<RunResult.STOPPED: 2>

In [84]:
schunk_wsg50_model_url = FindResourceOrThrow(
    "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf"
)

In [85]:
schunk_wsg50_model_url

'/home/dema/PycharmProjects/robot_manipulator/lib/python3.10/site-packages/pydrake/common/../lib/../share/drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf'

In [87]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(file_name=schunk_wsg50_model_url)
visualizer.Run(loop_once=False)

Click 'Stop Running' or press Esc to quit


<RunResult.STOPPED: 2>

# Model DEF sdf

In [68]:
cylinder_sdf = """
<?xml version="1.0"?>
<sdf version="1.7">
    <model name="cylinder">
        <pose>0 0 0 0 0 0</pose>
        <link name="cylinder_link">
            <inertial>
                <mass>1.0</mass>
                <inertia>
                  <ixx>0.005833</ixx>
                  <ixy>0.0</ixy>
                  <ixz>0.0</ixz>
                  <iyy>0.005833</iyy>
                  <iyz>0.0</iyz>
                  <izz>0.005</izz>
                </inertia>
            </inertial>

            <collision name="collision">
                <geometry>
                    <cylinder>
                        <radius>0.1</radius>
                        <length>0.2</length>
                    </cylinder>
                </geometry>
            </collision>

            <visual name="visual">
                <geometry>
                    <cylinder>
                        <radius>0.1</radius>
                        <length>0.2</length>                    
                    </cylinder>
                </geometry>
                <material>
                    <diffuse>1.0 1.0 1.0 1.0</diffuse>
                </material>
            </visual>
        </link>
    </model>
</sdf>
"""

visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(cylinder_sdf, 'sdf')
visualizer.Run(loop_once=True)

<RunResult.LOOP_ONCE: 1>

In [95]:
path = '../robot_models/planar_2R.sdf'
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelFromFile(path)
visualizer.Run(loop_once=False)

Click 'Stop Running' or press Esc to quit


<RunResult.STOPPED: 2>

In [ ]:
full_2r_directives = '''
directives: 
- add_model: 
    name: 2R_planar_robot
    file: file:///home/dema/PycharmProjects/robot_manipulator/robot_models/planar_2R.sdf
    default_joint_positions: 
        q1: [0]
        q2: [0]


- add_model: 
    name: end_effector
    file: file:///home/dema/PycharmProjects/robot_manipulator/lib/python3.10/site-packages/pydrake/common/../lib/../share/drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld: 
    parent: 2R_planar_robot::l2
    child: end_effector::body_frame
    X_PC: 
        translation: [0, 0, -0.25]
        rotation: !Rpy {deg: [-90, 0, 0]}
'''
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(full_2r_directives, '.dmd.yaml')
visualizer.Run(loop_once=False)


Click 'Stop Running' or press Esc to quit


In [111]:
import urllib.parse

def path_to_uri(path):
    """Convert a local file path to a file URI."""
    # Use the 'file://' scheme for local file URIs
    # Encode the path to handle spaces and other special characters
    return urllib.parse.urljoin('file:', urllib.parse.quote(path))

In [112]:
path_to_uri('../robot_models/planar_2R.sdf')

'file:///robot_models/planar_2R.sdf'